# i18

The i18n library provides you an easier way to manage languages with grammars and lexers.

## Replace a string given a grammar and a replace function


```python
sub(grammar, fn_replace, string)
```

In Lark syntax, where the grammar expects a group symbol, fn_replace is a function that replaces this group symbol; any other symbols remain unchanged. The string adheres to the rules of a Context-Free Grammar (CFG) rather than a regular expression (regex). 

In [4]:
from i18.parser import sub

sub("""
start: expression

expression: atom ("+"|"-"|"*"|"/") expression | atom

atom: SIGNED_NUMBER -> group
    | "(" expression ")"

%import common.SIGNED_NUMBER
%import common.WS
%ignore WS

""", lambda token: str(int(token)%2), "(4+(5+6)+5)+3")

'(0+(1+0)+1)+1'

## Translate your blade templates

When you have embedded code in HTML, you can translate both without losing any information

In [32]:
from i18.grammars import html_grammar
from i18.parser import sub
from deep_translator import (GoogleTranslator,DeeplTranslator)

translator = GoogleTranslator(source='auto', target='de')

new_text = sub(html_grammar, lambda token: translator.translate(text=token), """
<span class="flex">
     @if($step == 1)
            <span class="text-blue-600">Hello <p>World {{$x->g}}</p></span>
     @else
            <span class="text-green-500">You're welcome</span>
     @endif
</span>
""")
print(new_text)


<span class="flex">
     @if($step == 1)
            <span class="text-blue-600">Hallo<p>Welt {{$x->g}}</p></span>
     @else
            <span class="text-green-500">Gern geschehen</span>
     @endif
</span>



In [6]:
from i18.grammars import html_grammar
from i18.parser import sub
from deep_translator import (GoogleTranslator,DeeplTranslator)

translator = GoogleTranslator(source='auto', target='de')

new_text = sub(html_grammar, lambda token: "0", """<p>{{$x->g}}</p>""")
print(new_text)

<p>0</p>


# Search in string based on Grammar

In [28]:
from i18.parser import search
from i18.grammars import json_grammar, sub_in_grammar

search(sub_in_grammar(json_grammar, {"GROUP": r"/[\w_.]+@[\w_.]+\.[\w-]{2,4}/"}), 
       '{"key": ["abdiel@apimarket.mx"], "abdiel2@apimarket.mx": "2"}')

[Token('__ANON_1', 'abdiel@apimarket.mx')]

# Lexer

A lexer converts a sequence of characters into a sequence of tokens, but these tokens have a meaning associated with them. This task is quite similar to named-entity recognition (yes, you could use Spacy), however, a classic lexer does the job.

In [4]:
from i18.lexer import Lexer
lexer = Lexer()
[token for token in lexer.tokenize("""{{!!$person->name!!}}""")]

[LexToken(BLADE_ECHO,'{{!!$person->name!!}}',1,0)]

# Apply i18

In [14]:
from i18 import apply_i18

new_text, translations = apply_i18("""
    <div>Hello {{$x->g}} 
         <p>World</p>
    </div>
""")
print(new_text)


    <div>{{__("hello")}} {{$x->g}}<p>{{__("world")}}</p>
    </div>



In [15]:
translations

{'es': {'hello': 'Hola', 'world': 'Mundo'},
 'en': {'hello': 'Hello', 'world': 'World'}}

# Tests

In [2]:
from i18.parser import parse, print_pretty
from i18.grammars import html_grammar

assert parse(html_grammar, "<div>1</div>").pretty() == "start\n  element\n    tag\tdiv\n    group\t1\n    tag\tdiv\n"
assert parse(html_grammar, '<div @id.g="1">1</div>').pretty() == 'start\n  element\n    tag\tdiv\n    attribute\n      @\n      id.g\n      ="1"\n    group\t1\n    tag\tdiv\n'
assert parse(html_grammar, "<div @id.g='1'>1</div>").pretty() == "start\n  element\n    tag\tdiv\n    attribute\n      @\n      id.g\n      ='1'\n    group\t1\n    tag\tdiv\n"
assert parse(html_grammar, "<div :id.g.x.{{ $x}}='1'>1</div>").pretty() == "start\n  element\n    tag\tdiv\n    attribute\n      :\n      id.g.x.{{ $x}}\n      ='1'\n    group\t1\n    tag\tdiv\n"
assert parse(html_grammar, """
  <linearGradient id=":R2m96:" x1="11.5" y1="18" x2="36" y2="15.5"
                                                gradientUnits="userSpaceOnUse">
                                    <stop offset=".194" stop-color="#fff"></stop>
                                    <stop offset="1" stop-color="#6692F1"></stop>
                                </linearGradient>

""").pretty() == 'start\n  element\n    tag\tlinearGradient\n    attribute\n      id\n      =":R2m96:"\n    attribute\n      x1\n      ="11.5"\n    attribute\n      y1\n      ="18"\n    attribute\n      x2\n      ="36"\n    attribute\n      y2\n      ="15.5"\n    attribute\n      gradientUnits\n      ="userSpaceOnUse"\n    element\n      tag\tstop\n      attribute\n        offset\n        =".194"\n      attribute\n        stop-color\n        ="#fff"\n      tag\tstop\n    element\n      tag\tstop\n      attribute\n        offset\n        ="1"\n      attribute\n        stop-color\n        ="#6692F1"\n      tag\tstop\n    tag\tlinearGradient\n'
assert parse(html_grammar, """
  @error($parameter->name) <span  class="text-sm text-red-500"> {{$message}}</span> @enderror
""").pretty() == 'start\n  element\n    args\n      expression\n        function_call\n          variable\tparameter\n          name\n    start\n      element\n        tag\tspan\n        attribute\n          class\n          ="text-sm text-red-500"\n        group\t {{$message}}\n        tag\tspan\n'

AssertionError: 

In [3]:
print_pretty(html_grammar, """
@if($open)
            @isset($response['component'])
                @includeIf('livewire.app.services.form.'.$response['component'])
            @endisset
@endif
""")

start
  element
    ($open)
    start
      element
        args
          expression
            variable	response
            expression
              literal	'component'
        start
          element
            blade_expression	@includeIf('livewire.app.services.form.'.$response['component'])

